In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = '/home/husein/t5/prepare/mesolitica-tpu.json'

In [2]:
import malaya_speech.train.model.conformer as conformer
import malaya_speech.train.model.transducer as transducer
import malaya_speech
import tensorflow as tf
import numpy as np
import json
from glob import glob
import pandas as pd






The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [3]:
subwords = malaya_speech.subword.load('transducer-singlish.subword')

In [4]:
featurizer = malaya_speech.tf_featurization.STTFeaturizer(
    normalize_per_feature = True
)

In [5]:
n_mels = 80
sr = 16000
maxlen = 18
minlen_text = 1

def mp3_to_wav(file, sr = sr):
    audio = AudioSegment.from_file(file)
    audio = audio.set_frame_rate(sr).set_channels(1)
    sample = np.array(audio.get_array_of_samples())
    return malaya_speech.astype.int_to_float(sample), sr


def generate(file):
    print(file)
    with open(file) as fopen:
        audios = json.load(fopen)
    for i in range(len(audios)):
        try:
            audio = audios[i][0]
            wav_data, _ = malaya_speech.load(audio, sr = sr)

            if (len(wav_data) / sr) > maxlen:
                # print(f'skipped audio too long {audios[i]}')
                continue

            if len(audios[i][1]) < minlen_text:
                # print(f'skipped text too short {audios[i]}')
                continue

            t = malaya_speech.subword.encode(
                subwords, audios[i][1], add_blank = False
            )
            back = np.zeros(shape=(2000,))
            front = np.zeros(shape=(200,))
            wav_data = np.concatenate([front, wav_data, back], axis=-1)

            yield {
                'waveforms': wav_data,
                'targets': t,
                'targets_length': [len(t)],
            }
        except Exception as e:
            print(e)


def preprocess_inputs(example):
    s = featurizer.vectorize(example['waveforms'])
    mel_fbanks = tf.reshape(s, (-1, n_mels))
    length = tf.cast(tf.shape(mel_fbanks)[0], tf.int32)
    length = tf.expand_dims(length, 0)
    example['inputs'] = mel_fbanks
    example['inputs_length'] = length
    example.pop('waveforms', None)
    example['targets'] = tf.cast(example['targets'], tf.int32)
    example['targets_length'] = tf.cast(example['targets_length'], tf.int32)
    return example


def get_dataset(
    file,
    batch_size = 3,
    shuffle_size = 20,
    thread_count = 24,
    maxlen_feature = 1800,
):
    def get():
        dataset = tf.data.Dataset.from_generator(
            generate,
            {
                'waveforms': tf.float32,
                'targets': tf.int32,
                'targets_length': tf.int32,
            },
            output_shapes = {
                'waveforms': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
                'targets_length': tf.TensorShape([None]),
            },
            args = (file,),
        )
        dataset = dataset.prefetch(tf.contrib.data.AUTOTUNE)
        dataset = dataset.map(
            preprocess_inputs, num_parallel_calls = thread_count
        )
        dataset = dataset.padded_batch(
            batch_size,
            padded_shapes = {
                'inputs': tf.TensorShape([None, n_mels]),
                'inputs_length': tf.TensorShape([None]),
                'targets': tf.TensorShape([None]),
                'targets_length': tf.TensorShape([None]),
            },
            padding_values = {
                'inputs': tf.constant(0, dtype = tf.float32),
                'inputs_length': tf.constant(0, dtype = tf.int32),
                'targets': tf.constant(0, dtype = tf.int32),
                'targets_length': tf.constant(0, dtype = tf.int32),
            },
        )
        return dataset

    return get

In [6]:
dev_dataset = get_dataset('test-set-imda.json', batch_size = 3)()
features = dev_dataset.make_one_shot_iterator().get_next()
features

Instructions for updating:
Use `for ... in dataset:` to iterate over a dataset. If using `tf.estimator`, return the `Dataset` object directly from your input function. As a last resort, you can use `tf.compat.v1.data.make_one_shot_iterator(dataset)`.


{'targets': <tf.Tensor 'IteratorGetNext:2' shape=(?, ?) dtype=int32>,
 'targets_length': <tf.Tensor 'IteratorGetNext:3' shape=(?, ?) dtype=int32>,
 'inputs': <tf.Tensor 'IteratorGetNext:0' shape=(?, ?, 80) dtype=float32>,
 'inputs_length': <tf.Tensor 'IteratorGetNext:1' shape=(?, ?) dtype=int32>}

In [7]:
training = True

In [8]:
config = malaya_speech.config.conformer_small_encoder_config
config['dropout'] = 0.0
conformer_model = conformer.Model(
    kernel_regularizer = None, bias_regularizer = None, **config
)
decoder_config = malaya_speech.config.conformer_small_decoder_config
decoder_config['embed_dropout'] = 0.0
transducer_model = transducer.rnn.Model(
    conformer_model, vocabulary_size = subwords.vocab_size, **decoder_config
)
targets_length = features['targets_length'][:, 0]
v = tf.expand_dims(features['inputs'], -1)
z = tf.zeros((tf.shape(features['targets'])[0], 1), dtype = tf.int32)
c = tf.concat([z, features['targets']], axis = 1)

logits = transducer_model([v, c, targets_length + 1], training = training)

Instructions for updating:
If using Keras pass *_constraint arguments to layers.

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [9]:
decoded = transducer_model.greedy_decoder(v, features['inputs_length'][:, 0], training = training)
decoded

<tf.Tensor 'while/Exit_1:0' shape=(?, ?) dtype=int32>

In [10]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
var_list = tf.get_collection(tf.GraphKeys.GLOBAL_VARIABLES)
saver = tf.train.Saver(var_list = var_list)
saver.restore(sess, 'asr-small-conformer-transducer-singlish/model.ckpt-670000')

INFO:tensorflow:Restoring parameters from asr-small-conformer-transducer-singlish/model.ckpt-670000


In [11]:
wer, cer = [], []
index = 0
while True:
    try:
        r = sess.run([decoded, features['targets']])
        for no, row in enumerate(r[0]):
            d = malaya_speech.subword.decode(subwords, row[row > 0])
            t = malaya_speech.subword.decode(subwords, r[1][no])
            wer.append(malaya_speech.metrics.calculate_wer(t, d))
            cer.append(malaya_speech.metrics.calculate_cer(t, d))
        index += 1
    except Exception as e:
        break

b'test-set-imda.json'


In [12]:
np.mean(wer), np.mean(cer)

(0.1277107594947978, 0.07039536467199015)

In [13]:
for no, row in enumerate(r[0]):
    d = malaya_speech.subword.decode(subwords, row[row > 0])
    t = malaya_speech.subword.decode(subwords, r[1][no])
    print(no, d)
    print(t)
    print()

0 the health ministry encourages all households to update their information so these subsidies can be calculated accurately
the health ministry encourages all households to update their information so these subsidies can be calculated accurately

1 bus bridging will be deployed for the affected ratch for the duration of the suspension
bus bridging will be deployed for the affected stretch for the duration of the suspension

